In [1]:
# https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/wide_deep_movielens.ipynb
# not implement cross column

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
from evals import *
from models import WideAndDeep

import warnings
warnings.filterwarnings('ignore')

In [3]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.utils import to_categorical

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

## Load

In [4]:
df = load_data('../data/ml-100k/u.data', threshold=3)

uuid = df['userId'].unique()
uiid = df['movieId'].unique()


In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df['userId'].values)

## Model

In [6]:
class WideAndDeep(keras.Model):
    def __init__(self, u_dim, i_dim, u_emb_dim=4, i_emb_dim=4):
        super(WideAndDeep, self).__init__()
        
        self.u_dim = u_dim
        self.i_dim = i_dim
        self.u_emb_dim = u_emb_dim
        self.i_emb_dim = i_emb_dim
        
        self.deep_model = self.build_deep_model()
        self.wide_model = self.build_wide_model()


    def compile(self, wide_optim, deep_optim, loss_fn):
        super(WideAndDeep, self).compile()
        self.wide_optim = wide_optim
        self.deep_optim = deep_optim
        self.loss_fn = loss_fn
    
    def build_deep_model(self):
        u_input = Input(shape=(1, ))
        i_input = Input(shape=(1, ))

        u_emb = Flatten()(Embedding(self.u_dim, self.u_emb_dim, input_length=u_input.shape[1])(u_input))
        i_emb = Flatten()(Embedding(self.i_dim, self.i_emb_dim, input_length=i_input.shape[1])(i_input))

        concat = Concatenate()([u_emb, i_emb])
        
        h = Dense(256, activation='relu')(concat)
        h = Dense(128, activation='relu')(h)
        h = Dense(64, activation='relu')(h)
        h = Dropout(0.2)(h)

        out = Dense(1)(h)
        
        return Model([u_input, i_input], out, name='DeepModel')
    
    def build_wide_model(self):
        u_input = Input(shape=(self.u_dim, ))
        i_input = Input(shape=(self.i_dim, ))

        concat = Concatenate()([u_input, i_input])
        
        out = Dense(1)(concat)
        
        return Model([u_input, i_input], out, name='WideModel')
        
    
    def train_step(self, data):
        X, y = data
        user, item, user_ohe, item_ohe = X
        
        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            wide_logit = self.wide_model([user_ohe, item_ohe])
            deep_logit = self.deep_model([user, item])
            logit = 0.5*(wide_logit + deep_logit)
            
            loss = self.loss_fn(y, logit)
            
        wide_grads = tape1.gradient(loss, self.wide_model.trainable_weights)
        self.wide_optim.apply_gradients(zip(wide_grads, self.wide_model.trainable_weights))
        
        deep_grads = tape2.gradient(loss, self.deep_model.trainable_weights)
        self.deep_optim.apply_gradients(zip(deep_grads, self.deep_model.trainable_weights))
        
        return {'loss': loss}
    
    def call(self, data):
        user, item, user_ohe, item_ohe = data
        wide_logit = self.wide_model([user_ohe, item_ohe])
        deep_logit = self.deep_model([user, item])
        return 0.5*(wide_logit + deep_logit)


## Train

In [7]:
wnd = WideAndDeep(len(uuid), len(uiid))
wnd.compile(
    optimizers.Adam(1e-3),
    optimizers.Adam(1e-3),
    losses.BinaryCrossentropy(from_logits=True)
           )

hist = wnd.fit([train['userId'].values, train['movieId'].values, to_categorical(train['userId']), to_categorical(train['movieId'])],
                   train['rating'].values,
                   shuffle=True,
                   epochs=10,
                   validation_split=0.1
                  )

Epoch 1/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5965 - val_loss: 0.0000e+00
Epoch 2/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5550 - val_loss: 0.0000e+00
Epoch 3/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5440 - val_loss: 0.0000e+00
Epoch 4/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5373 - val_loss: 0.0000e+00
Epoch 5/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5324 - val_loss: 0.0000e+00
Epoch 6/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5253 - val_loss: 0.0000e+00
Epoch 7/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5189 - val_loss: 0.0000e+00
Epoch 8/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5125 - val_loss: 0.0000e+00
Epoch 9/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5060 - val_loss: 0.0000e+00
Epoch 10/10
2391/2391 [======================

## Eval

In [8]:
pred = wnd.predict([test['userId'].values, test['movieId'].values, to_categorical(test['userId'], len(uuid)), to_categorical(test['movieId'], len(uiid))])
# np.mean(np.square(test['rating'].values, pred.flatten()))


In [9]:
# accuracy
np.sum(np.where(pred>0, 1, 0).flatten() == test['rating'].values) / len(pred)

0.7208666666666667

In [10]:
from sklearn.metrics import precision_score, recall_score,  roc_auc_score

print(roc_auc_score(test['rating'].values, pred.flatten()))
print(precision_score(test['rating'].values, np.where(pred>0, 1, 0).flatten()))
print(recall_score(test['rating'].values, np.where(pred>0, 1, 0).flatten()))

0.7877736309877519
0.7322091062394603
0.7830007213272421
